(+) This notebook serves to compute intermediate  weekly statistics features. These weekly features will be aggregated later as final features for modeling steps. 
(++) The dataset covered transaction in 2016 consists of 52 weeks, so we compute basic statistics for each week among these weeks


In [3]:
import numpy as np
import pickle 
import pandas as pd

import datetime
WORK_PATH = "/Users/welcome/Google Drive (cuong.tranus@gmail.com)/research/internship/fraud-detection"
feat_dict = pickle.load(open(WORK_PATH + "/results/features_category.pkl" ,"rb"))

numeric_cols = feat_dict['numeric_cols']
cat_cols = feat_dict['cat_cols']

print(cat_cols)
print(numeric_cols)

['transactionDateTime', 'merchantName', 'posEntryMode', 'posConditionCode', 'merchantCategoryCode', 'currentExpDate', 'accountOpenDate', 'dateOfLastAddressChange', 'transactionType']
['accountNumber', 'customerId', 'creditLimit', 'availableMoney', 'transactionAmount', 'cardCVV', 'enteredCVV', 'cardLast4Digits', 'currentBalance', 'cardPresent', 'isFraud']


In [5]:
pd00 = pd.read_csv(WORK_PATH + "/data/processed_data.csv")
print(pd00.shape)

(786363, 26)


In [29]:
pd00['transactionType'].value_counts()

PURCHASE                745193
REVERSAL                 20303
ADDRESS_VERIFICATION     20169
Name: transactionType, dtype: int64

In [11]:
## Extract date info
def get_week(date):
    """
    get week number based on date
    return an integer number between 0 and 51
    """
    y,m, d = date.split("-")
    week = datetime.date(int(y), int(m), int(d)).isocalendar()[1]
    if week >52:
        # for some reasons get_week('2016-01-01') return 53 but not zero 
        return 0
    else:
        return week

print(get_week('2016-01-01'))
print(get_week('2016-12-31'))
print(get_week('2016-01-07'))

pd00['date'] = pd00['transactionDateTime'].apply(lambda x: x.split("T")[0]).astype(str)
pd00['week'] = pd00['date'].apply(get_week)


0
52
1


### I. Intermediate weekly features for numetric columns

In [30]:
agg_func = {col :[np.sum, np.min, np.max] for col in ['creditLimit','availableMoney','transactionAmount','currentBalance']}
agg_func['customerId']= np.size
pd01 = pd00.groupby(['accountNumber','week']).agg(agg_func).reset_index()


In [31]:
pd01.head(2)

accountNumber week creditLimit               availableMoney            \
                             sum   amin   amax            sum      amin   
0     100088067    2      150000  50000  50000      149543.31  49748.44   
1     100088067    5      100000  50000  50000       99933.93  49933.93   

           transactionAmount                currentBalance               \
      amax               sum   amin    amax            sum amin    amax   
0  50000.0            630.23  46.43  378.67         456.69  0.0  251.56   
1  50000.0            207.67  66.07  141.60          66.07  0.0   66.07   

  customerId  
        size  
0          3  
1          2

In [32]:
pd01.columns = ["accountNumber", "week"] + ["_".join(col) for col in pd01.columns.values[2:-1]] + ['num_trans']
pd01.head(2)

,accountNumber,week,creditLimit_sum,creditLimit_amin,creditLimit_amax,availableMoney_sum,availableMoney_amin,availableMoney_amax,transactionAmount_sum,transactionAmount_amin,transactionAmount_amax,currentBalance_sum,currentBalance_amin,currentBalance_amax,num_trans
0,100088067,2,150000,50000,50000,149543.31,49748.44,50000.0,630.23,46.43,378.67,456.69,0.0,251.56,3
1,100088067,5,100000,50000,50000,99933.93,49933.93,50000.0,207.67,66.07,141.60,66.07,0.0,66.07,2


In [37]:
pd_list = []
agg_func_lb4w_dict = {}
for col in pd01.columns.tolist():
    if 'min' in col:
        agg_func_lb4w_dict[col] = np.min
    elif 'max' in col:
        agg_func_lb4w_dict[col] = np.max
    elif 'sum' in col or 'num_trans' in col: 
        agg_func_lb4w_dict[col] = np.sum
        
        

In [44]:
pd_list = []
for week in range(4,52):
    temp_pd = pd01[(pd01['week']>=week-4) & (pd01['week']<=week-1)]
    group_pd = temp_pd.groupby('accountNumber').agg(agg_func_lb4w_dict).reset_index()
    group_pd['week'] = week
    pd_list.append(group_pd)
    

In [45]:
pd02 = pd.concat(pd_list)
pd02.head(4)

,accountNumber,creditLimit_sum,creditLimit_amin,creditLimit_amax,availableMoney_sum,availableMoney_amin,availableMoney_amax,transactionAmount_sum,transactionAmount_amin,transactionAmount_amax,currentBalance_sum,currentBalance_amin,currentBalance_amax,num_trans,week
0,100088067,150000,50000,50000,149543.31,49748.44,50000.0,630.23,46.43,378.67,456.69,0.0,251.56,3,4
1,100328049,25000,5000,5000,24144.81,4605.10,5000.0,438.64,0.00,242.03,855.19,0.0,394.90,5,4
2,100663626,60000,20000,20000,59806.53,19822.85,20000.0,223.91,16.32,160.83,193.47,0.0,177.15,3,4
3,100737756,3500,250,250,-284.62,-535.03,250.0,4470.98,0.00,693.50,3784.62,0.0,785.03,14,4


### II. Intermediate categorical features